---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [87]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
    
def blight_model():
    
    train = pd.read_csv('train.csv', encoding = "ISO-8859-1")
    test = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    addresses = pd.read_csv('addresses.csv', encoding = "ISO-8859-1")
    
    train = pd.merge(train, addresses, how='inner', left_on='ticket_id', right_on='ticket_id')
    test = pd.merge(test, addresses, how='inner', left_on='ticket_id', right_on='ticket_id')
    
    train = train.dropna(subset=['compliance'])
    train['compliance'] = train['compliance'].astype(int)
    
    
    convert_columns={'country': 'category',
                     'non_us_str_code': 'category',
                     'compliance': 'category',
                     'state': 'category',
                     'zip_code': 'category'
                    }
    
    for df in [test, train]:
        for col, col_type in convert_columns.items():
            if col in df:
                if col_type == 'category':
                    df[col] = df[col].replace(np.nan, "NA", regex=True).astype(col_type)
                elif col_type == 'int':
                    df[col] = df[col].replace(np.nan, 0, regex=True).astype(col_type)
    
   
    common_exclude = ['agency_name', 'inspector_name', 'mailing_address_str_number',
                           'violator_name', 'violation_street_number', 'violation_street_name',
                           'mailing_address_str_name', 'address', 'admin_fee', 'violation_zip_code',
                           'state_fee', 'late_fee', 'ticket_issued_date', 'hearing_date', 'violation_description',
                           'fine_amount', 'clean_up_cost', 'disposition', 'grafitti_status',
                           'violation_code', 'city']
    train_exclude = ['payment_status', 'payment_date', 'balance_due', 'payment_amount'] + common_exclude
    train = train.drop(train_exclude, axis=1).set_index('ticket_id')
    test = test.drop(common_exclude, axis=1).set_index('ticket_id')
    y_train = train['compliance']  
    X_train_cols_to_drop = ['compliance', 'compliance_detail', 'collection_status']
    train = train.drop(X_train_cols_to_drop, axis=1)
    
    cat_columns = train.select_dtypes(['category']).columns
    for df in [test, train]:
        df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
        
    X_train = train.copy()
    print(len(X_train.columns))
    # Random Forest
    rfclf = RandomForestClassifier(n_jobs=-1, oob_score = True,random_state = 99) 
    grid_values = { "n_estimators" : [50, 100, 400],"max_depth" : [10,20, 40],"max_features" : [2,3,4]}
    rand_rfclf_auc = RandomizedSearchCV(rfclf, grid_values, scoring='roc_auc', n_iter=10)
    rand_rfclf_auc.fit(X_train, y_train)
    #grid_rfclf_auc = GridSearchCV(rfclf, param_grid = grid_values, scoring = 'roc_auc')
    #grid_rfclf_auc.fit(X_train, y_train)
    rfprobs = rand_rfclf_auc.predict_proba(test)[:, 1]
    
    print("RandomForestClassifier",rfprobs)
    
    # Boosting
    bclf = GradientBoostingClassifier(random_state = 99)
    grid_values = {'learning_rate': [0.01, 0.001, 0.1], 'max_depth': [1, 2, 3]}
    grid_clf_auc = GridSearchCV(bclf, param_grid = grid_values, scoring = 'roc_auc')
    grid_clf_auc.fit(X_train, y_train)
    bprobs = grid_clf_auc.predict_proba(test)[:, 1]
    print("GradientBoostingClassifier",bprobs)
    
    # Neural Network
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(test)
    
    nnclf = MLPClassifier(hidden_layer_sizes = [5, 4,3], alpha = 5,
                       random_state = 99, solver='lbfgs', verbose=0)
    nnclf.fit(X_train_scaled, y_train)
    nnprobs = nnclf.predict_proba(X_test_scaled)[:,1]
    print("MLPClassifier",nnprobs)
    
    print("Combined Classification",(nnprobs+bprobs+rfprobs)/3)
    result = pd.Series((nnprobs+bprobs+rfprobs)/3, index=test.index)

    return result

blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


6
RandomForestClassifier [ 0.13051758  0.03511868  0.14891247 ...,  0.14891247  0.15402162
  0.21287162]
GradientBoostingClassifier [ 0.15570146  0.07561872  0.17318216 ...,  0.17318216  0.17318216
  0.47541994]
MLPClassifier [ 0.07256118  0.07256118  0.07256118 ...,  0.07256118  0.07256118
  0.07256118]
Combined Classification [ 0.11959341  0.06109953  0.13155194 ...,  0.13155194  0.13325499
  0.25361758]


ticket_id
284932    0.119593
285362    0.061100
285361    0.131552
285338    0.120298
285346    0.131202
285345    0.123735
285347    0.139711
285342    0.248476
285530    0.183444
284989    0.067273
285344    0.142843
285343    0.062693
285340    0.061100
285341    0.144172
285349    0.168812
285348    0.153999
284991    0.095355
285532    0.071690
285406    0.073875
285001    0.073654
285006    0.063007
285405    0.068484
285337    0.212001
285496    0.147718
285497    0.120268
285378    0.058289
285589    0.091820
285585    0.119507
285501    0.131507
285581    0.061545
            ...   
376367    0.077931
376366    0.067166
376362    0.069691
376363    0.119568
376365    0.076856
376364    0.066725
376228    0.066725
376265    0.069376
376286    0.295490
376320    0.066725
376314    0.066725
376327    0.259652
376385    0.259652
376435    0.077317
376370    0.254919
376434    0.144075
376459    0.131553
376478    0.043381
376473    0.066659
376484    0.067911
376482    0.108447
37